# IBIT Backtest: AMMA vs Trend + Mining + OU

This notebook curates IBIT and cleaned crypto mining-cost data, aligns dates, and compares:
- **AMMA** strategy (fit to `IBIT_Data.csv`)
- **Buy & Hold** on IBIT
- **Trend + Mining + OU** ensemble strategy

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from Crypto.Backtest.amma import amma_from_ibit_csv
from Crypto.Models.trend import trend_signal
from Crypto.Models.mining import mining_signal
from Crypto.Models.ou import ou_signal

plt.style.use('seaborn-v0_8-darkgrid')

In [ ]:
IBIT_PATH = 'Crypto/Data/IBIT_Data.csv'
CLEANED_PATH = 'Crypto/Data/cleaned_crypto_data.csv'

ibit = pd.read_csv(IBIT_PATH)
cleaned = pd.read_csv(CLEANED_PATH)

ibit['Date'] = pd.to_datetime(ibit['Date'], format='%m/%d/%y', errors='coerce')
ibit['IBIT_price'] = (
    ibit['Price']
    .astype(str)
    .str.replace(',', '', regex=False)
    .str.replace('$', '', regex=False)
    .astype(float)
)

cleaned['Date'] = pd.to_datetime(cleaned['Date'], errors='coerce')

curated = (
    ibit[['Date', 'IBIT_price']]
    .merge(cleaned[['Date', 'COST_TO_MINE']], on='Date', how='inner')
    .dropna(subset=['Date', 'IBIT_price', 'COST_TO_MINE'])
    .sort_values('Date')
    .reset_index(drop=True)
)

curated['ret'] = curated['IBIT_price'].pct_change().fillna(0.0)
curated.head(), curated[['Date','IBIT_price','COST_TO_MINE']].describe(include='all')

In [ ]:
# AMMA on IBIT
momentum_weights = {5: 0.25, 20: 0.35, 60: 0.40}
amma_df = amma_from_ibit_csv(
    ibit_csv_path=IBIT_PATH,
    momentum_weights=momentum_weights,
    threshold=0.0,
    long_enabled=True,
    short_enabled=False,
)

amma_curated = curated[['Date', 'ret']].merge(
    amma_df[['Date', 'amma_position']],
    on='Date',
    how='left'
).fillna({'amma_position': 0.0})

amma_curated['amma_ret'] = amma_curated['amma_position'] * amma_curated['ret']

In [ ]:
# Trend + Mining + OU using IBIT price and mining cost aligned by date
model_df = curated.copy()
model_df['BTC-USD_close'] = model_df['IBIT_price']

trend_pos = trend_signal(
    model_df,
    price_column='BTC-USD_close',
    fast_window=20,
    slow_window=128,
    long_only=True,
)

mining_pos = mining_signal(
    model_df,
    z_window=180,
    entry_z=0.5,
    exit_z=0.0,
    use_log_edge=True,
)

ou_pos = ou_signal(
    model_df['BTC-USD_close'],
    window=90,
    entry_z=1.5,
    exit_z=0.3,
    long_short=False,
    detrend_window=180,
)

combo_pos = (0.4 * trend_pos + 0.3 * mining_pos + 0.3 * ou_pos).clip(0.0, 1.5)

strategy = curated[['Date', 'ret']].copy()
strategy['trend_pos'] = trend_pos.values
strategy['mining_pos'] = mining_pos.values
strategy['ou_pos'] = ou_pos.values
strategy['combo_pos'] = combo_pos.values
strategy['combo_ret'] = strategy['combo_pos'] * strategy['ret']

In [ ]:
# Buy & Hold baseline
results = curated[['Date']].copy()
results['buy_hold_ret'] = curated['ret']
results['amma_ret'] = amma_curated['amma_ret'].values
results['combo_ret'] = strategy['combo_ret'].values

for c in ['buy_hold_ret', 'amma_ret', 'combo_ret']:
    results[f'{c}_equity'] = (1 + results[c]).cumprod()

results.tail()

In [ ]:
def perf_summary(returns, days=252):
    r = pd.Series(returns).dropna()
    if len(r) < 2:
        return {'CAGR': np.nan, 'Sharpe': np.nan, 'MaxDD': np.nan}

    equity = (1 + r).cumprod()
    years = len(r) / days
    cagr = equity.iloc[-1] ** (1 / years) - 1 if years > 0 else np.nan
    vol = r.std(ddof=1) * np.sqrt(days)
    sharpe = (r.mean() * days) / vol if vol > 0 else np.nan
    dd = equity / equity.cummax() - 1

    return {'CAGR': cagr, 'Sharpe': sharpe, 'MaxDD': dd.min()}

summary = pd.DataFrame({
    'Buy & Hold': perf_summary(results['buy_hold_ret']),
    'AMMA': perf_summary(results['amma_ret']),
    'Trend+Mining+OU': perf_summary(results['combo_ret']),
}).T

summary

In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(results['Date'], results['buy_hold_ret_equity'], label='Buy & Hold', linewidth=2)
plt.plot(results['Date'], results['amma_ret_equity'], label='AMMA', linewidth=2)
plt.plot(results['Date'], results['combo_ret_equity'], label='Trend + Mining + OU', linewidth=2)
plt.title('IBIT Equity Curve Comparison')
plt.xlabel('Date')
plt.ylabel('Equity (Start = 1.0)')
plt.legend()
plt.tight_layout()
plt.show()